In [1]:
#input the info needed to create a connection between sql server and this notebook using sqlalchemy
import getpass
mypasswd = getpass.getpass()
username = 'sebmb'
host = 'pgsql.dsa.lan'
database = 'dsa_student'

········


In [2]:
#create a connection and delete my password once connected
from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine

postgres_db = {'drivername': 'postgres',
               'username': username,
               'password': mypasswd,
               'host': host,
               'database' :database}

engine = create_engine(URL(**postgres_db), echo=True)

del mypasswd

In [3]:
#load in the file and clean up the data into a pandas dataframe
import pandas as pd

file = 'Chicago-Crime-Sample-2012.csv'

crime_df = pd.read_csv(file)
crime_df = crime_df.rename(columns={crime_df.columns[0]:'crime_id'})
crime_df = crime_df.drop(columns=['crime_id'])
crime_df.columns = [x.lower() for x in crime_df.columns]
crime_df.columns
crime_df = crime_df.rename(columns={'x coordinate':'x_coordinate', 'y coordinate':'y_coordinate', 'fbi code':'fbi_code',
                                   'case number': 'case_number', 'primary type' :'primary_type', 
                                    'location description' :'location_description', 'community area':'community_area',
                                   'updated on':'updated_on'})
crime_df.head()

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,10433096,HZ170962,1/1/2012 0:00,026XX N MC VICKER AVE,1562,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,RESIDENCE,True,False,...,29.0,19.0,17,NaN,NaN,2012,5/11/2016 15:48,NaN,NaN,NaN
1,10433124,HZ170983,1/1/2012 0:00,026XX N MC VICKER AVE,1544,SEX OFFENSE,SEXUAL EXPLOITATION OF A CHILD,RESIDENCE,True,False,...,29.0,19.0,17,NaN,NaN,2012,5/11/2016 15:48,NaN,NaN,NaN
2,10532867,HZ276514,1/1/2012 0:00,036XX S RHODES AVE,1563,SEX OFFENSE,CRIMINAL SEXUAL ABUSE,APARTMENT,False,False,...,4.0,35.0,17,NaN,NaN,2012,5/26/2016 15:51,NaN,NaN,NaN
3,10536876,HZ280873,1/1/2012 0:00,062XX S ROCKWELL ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,15.0,66.0,11,NaN,NaN,2012,5/27/2016 15:48,NaN,NaN,NaN
4,9581929,HX232501,1/1/2012 0:00,006XX W 66TH ST,1563,SEX OFFENSE,CRIMINAL SEXUAL ABUSE,RESIDENCE,False,True,...,6.0,68.0,17,NaN,NaN,2012,8/17/2015 15:03,NaN,NaN,NaN


In [8]:
#creating a row for the unique identifiers in the new tables to be added into the crime csv for future querying
#this is creating the location coordinate id which has a unique identifier for the location coordinate table
location_df = crime_df[['latitude','longitude','x_coordinate', 'y_coordinate','location']]
location_df['lc_id'] = location_df.groupby(location_df.columns.tolist(), sort=False).ngroup() +1
location_df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,latitude,longitude,x_coordinate,y_coordinate,location,lc_id
0,NaN,NaN,NaN,NaN,NaN,0
1,NaN,NaN,NaN,NaN,NaN,0
2,NaN,NaN,NaN,NaN,NaN,0
3,NaN,NaN,NaN,NaN,NaN,0
4,NaN,NaN,NaN,NaN,NaN,0


In [9]:
#creating a row for the unique identifiers in the new tables to be added into the crime csv for future querying
#this is creating the location description id which has a unique identifier for the location description table
location_des_df = crime_df[['location_description']]
location_des_df['ld_id'] = location_des_df.groupby(location_des_df.columns.tolist(), sort=False).ngroup() +1
location_des_df.head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,location_description,ld_id
0,RESIDENCE,1
1,RESIDENCE,1
2,APARTMENT,2
3,RESIDENCE,1
4,RESIDENCE,1
5,OTHER,3
6,RESIDENCE,1
7,RESIDENCE,1
8,RESIDENCE,1
9,RESIDENCE,1


In [10]:
#creating a row for the unique identifiers in the new tables to be added into the crime csv for future querying
#this is creating the location block id which has a unique identifier for the location block table
location_b_df = crime_df[['block']]
location_b_df['lb_id'] = location_b_df.groupby(location_b_df.columns.tolist(), sort=False).ngroup() +1
location_b_df.head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,block,lb_id
0,026XX N MC VICKER AVE,1
1,026XX N MC VICKER AVE,1
2,036XX S RHODES AVE,2
3,062XX S ROCKWELL ST,3
4,006XX W 66TH ST,4
5,020XX N CALIFORNIA AVE,5
6,012XX S WASHTENAW AVE,6
7,094XX S THROOP ST,7
8,069XX S UNION AVE,8
9,078XX S JEFFERY BLVD,9


In [11]:
#creating a row for the unique identifiers in the new tables to be added into the crime csv for future querying
#this is creating the location area id which has a unique identifier for the location area table
location_a_df = crime_df[['beat','district','ward','community_area']]
location_a_df['la_id'] = location_a_df.groupby(location_a_df.columns.tolist(), sort=False).ngroup() +1
location_a_df.head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,beat,district,ward,community_area,la_id
0,2512,25,29.0,19.0,1
1,2512,25,29.0,19.0,1
2,212,2,4.0,35.0,2
3,825,8,15.0,66.0,3
4,723,7,6.0,68.0,4
5,1414,14,35.0,22.0,5
6,1023,10,28.0,29.0,6
7,2222,22,21.0,73.0,7
8,732,7,6.0,68.0,8
9,414,4,8.0,43.0,9


In [ ]:
#add them back to the original df as their own columns
crime_df['la_id'] = location_a_df['la_id']
crime_df['lc_id'] = location_df['lc_id']
crime_df['lb_id'] = location_b_df['lb_id']
crime_df['ld_id'] = location_des_df['ld_id']

In [ ]:
#move the csv file from pandas to sql to query the results later in the lab
crime_df.to_sql('crime_df', # The table to load
          engine,             # The engine created above
          schema= username,   # The schema where the table lives, our pawprint
          if_exists='replace', # If the table is found, it would keep loading the end of table.
          index=False,        # Recall that panda data frame has a row index, so we need to ignore it
          chunksize=50) 

In [ ]:
#create the tables for the data from the chicago crime csv to go into
#these tables are made to normalize the dataset and reduce redundancy within the file
CREATE TABLE sebmb.location_area (
    la_id SERIAL,
    beat INT,
    district INT,
    ward INT,
    community_area INT,
    PRIMARY KEY(la_id)
);

In [ ]:
CREATE TABLE sebmb.location_block (
    lb_id SERIAL,
    block VARCHAR(200),
    PRIMARY KEY(lb_id)
);

In [ ]:
CREATE TABLE sebmb.location_description (
    ld_id SERIAL,
    location_description VARCHAR(50),
    PRIMARY KEY(ld_id)
);

In [ ]:
CREATE TABLE sebmb.location_coordinate (
    lc_id SERIAL,
    x_coordinate INT,
    y_coordinate INT,
    longitude FLOAT,
    latitude FLOAT,
    location VARCHAR(40),
    PRIMARY KEY(lc_id)
);

In [ ]:
CREATE TABLE sebmb.crime_code (
    iucr VARCHAR(10),
    fbi_code VARCHAR(10),
    description VARCHAR(30),
    primary_type VARCHAR(100),
    PRIMARY KEY(iucr,fbi_code)
);

In [ ]:
CREATE TABLE sebmb.crime_record (
    id INT,
    iucr VARCHAR(10),
    fbi_code VARCHAR(10),
    cl_id INT,
    case_number VARCHAR(20),
    arrest VARCHAR(5),
    domestic VARCHAR(5),
    crime_date VARCHAR(50),
    year INT,
    updated_on VARCHAR(50),
    PRIMARY KEY(id),
    FOREIGN KEY(iucr,fbi_code) REFERENCES crime_code(iucr,fbi_code),
    FOREIGN KEY(cl_id) REFERENCES crime_location(cl_id)
);

In [ ]:
CREATE TABLE sebmb.crime_location (
    cl_id SERIAL,
    la_id INT,
    lb_id INT,
    lc_id INT,
    ld_id INT,
    PRIMARY KEY(cl_id),
    FOREIGN KEY(la_id) REFERENCES location_area(la_id),
    FOREIGN KEY(lb_id) REFERENCES location_block(lb_id),
    FOREIGN KEY(lc_id) REFERENCES location_coordinate(lc_id),
    FOREIGN KEY(ld_id) REFERENCES location_description(ld_id)
);

In [ ]:
#transfer the data rows from the chicago crime csv into the given tables
INSERT INTO sebmb.crime_code 
SELECT DISTINCT c."IUCR", c."FBI Code", c."Description", c."Primary Type" FROM crime_df c;

In [ ]:
INSERT INTO sebmb.location_area (beat,district,ward,community_area)
SELECT DISTINCT c."Beat", c."District", c."Ward", c."Community Area" FROM crime_df c;

In [ ]:
INSERT INTO sebmb.location_coordinate (x_coordinate,y_coordinate,longitude,latitude,location)
SELECT DISTINCT c."X Coordinate", c."Y Coordinate", c."Longitude", c."Latitude", c."Location" FROM crime_df c;

In [ ]:
INSERT INTO sebmb.location_description (location_description)
SELECT DISTINCT c."Location Description" FROM crime_df c;

In [ ]:
INSERT INTO sebmb.location_block (block)
SELECT DISTINCT c."Block" FROM crime_df c;

In [ ]:
#a sample from each table to make sure the data was entered into the correct columns
SELECT * FROM sebmb.crime_code LIMIT 2;
 iucr | fbi_code |      description      | primary_type
------+----------+-----------------------+--------------
 5004 | 17       | ATT CRIM SEXUAL ABUSE | SEX OFFENSE
 1680 | 19       | OTHER                 | GAMBLING
(2 rows)

SELECT * FROM sebmb.location_area LIMIT 2;
 la_id | beat | district | ward | community_area
-------+------+----------+------+----------------
     1 | 1131 |       11 |   24 |             26
     2 | 1432 |       14 |    1 |             22
(2 rows)

SELECT * FROM sebmb.location_coordinate LIMIT 2;
 lc_id | x_coordinate | y_coordinate |  longitude   |  latitude   |           location
-------+--------------+--------------+--------------+-------------+-------------------------------
     1 |            0 |            0 | -91.68656568 |  36.6194464 | (36.619446395, -91.686565684)
     2 |      1094587 |      1945065 | -87.92736489 | 42.00607411 | (42.006074109, -87.927364891)
(2 rows)

SELECT * FROM sebmb.location_description LIMIT 5;
 ld_id |             location_description
-------+----------------------------------------------
     1 |
     2 | TAXICAB
     3 | FEDERAL BUILDING
     4 | OTHER
     5 | CHA APARTMENT
(5 rows)

SELECT * FROM sebmb.location_block LIMIT 2;
 lb_id |        block
-------+----------------------
     1 | 036XX W WABANSIA AVE
     2 | 006XX W 115TH ST
(2 rows)

In [ ]:
#get the data types from each table
SELECT DATA_TYPE from INFORMATION_SCHEMA.COLUMNS where
table_schema = 'sebmb' and table_name = 'location_area';

data_type
-----------
 integer
 integer
 integer
 integer
 integer

    
SELECT DATA_TYPE from INFORMATION_SCHEMA.COLUMNS where
table_schema = 'sebmb' and table_name = 'location_block';

     data_type
-------------------
 integer
 character varying
(2 rows)

SELECT DATA_TYPE from INFORMATION_SCHEMA.COLUMNS where
table_schema = 'sebmb' and table_name = 'location_description';

  data_type
-------------------
 integer
 character varying
(2 rows)

SELECT DATA_TYPE from INFORMATION_SCHEMA.COLUMNS where
table_schema = 'sebmb' and table_name = 'location_coordinate';

  data_type
-------------------
 integer
 integer
 integer
 double precision
 double precision
 character varying
(6 rows)

SELECT DATA_TYPE from INFORMATION_SCHEMA.COLUMNS where
table_schema = 'sebmb' and table_name = 'crime_code';

   data_type
-------------------
 character varying
 character varying
 character varying
 character varying
(4 rows)

SELECT DATA_TYPE from INFORMATION_SCHEMA.COLUMNS where
table_schema = 'sebmb' and table_name = 'crime_location';

 data_type
-----------
 integer
 integer
 integer
 integer
 integer
(5 rows)

SELECT DATA_TYPE from INFORMATION_SCHEMA.COLUMNS where
table_schema = 'sebmb' and table_name = 'crime_record';

  data_type
-------------------
 integer
 character varying
 character varying
 integer
 character varying
 character varying
 character varying
 character varying
 integer
 character varying
(10 rows)

In [ ]:
#get the row count from each table
SELECT COUNT(*) FROM sebmb.location_area;

 count
-------
  1024
(1 row)

SELECT COUNT(*) FROM sebmb.location_block;

 count
-------
 28341
(1 row)

SELECT COUNT(*) FROM sebmb.location_description;

 count
-------
    99
(1 row)

SELECT COUNT(*) FROM sebmb.location_coordinate;

 count
--------
 162287
(1 row)

SELECT COUNT(*) FROM sebmb.crime_code;

 count
-------
   319
(1 row)

SELECT COUNT(*) FROM sebmb.crime_location;

 count
--------
 243426
(1 row)

SELECT COUNT(*) FROM sebmb.crime_record;

 count
--------
 334715
(1 row)

In [ ]:
#this query is where the crime occured, how many times it occured and if it ended in an arrest. 
#After looking at this query, I think it might have been better to limit the amount to the top 10 places a 
#crime occured instead of having each place. This is a good thing to not that the airport is where most crimes happen.

SELECT ld.location_description, count(*), cr.arrest
FROM sebmb.location_description ld JOIN sebmb.crime_location cl USING(ld_id)
JOIN sebmb.crime_record cr USING(cl_id)
GROUP BY (location_description, arrest)
ORDER BY count(*) DESC
LIMIT 5;

    location_description               | count | arrest
-------------------------------------------------+-------+--------
 AIRPORT EXTERIOR - NON-SECURE AREA              | 61679 | false
                                                 | 42626 | false
 AIRPORT TERMINAL MEZZANINE - NON-SECURE AREA    | 32395 | false
 TAXICAB                                         | 27875 | false
 AIRPORT EXTERIOR - NON-SECURE AREA              | 22323 | true

In [ ]:
#this query shows the top 10 blocks with the highest crime for the year 2012. This could be helpful 
#when looking to protect streets with more police. 

SELECT lb.block, count(*), cr.year
FROM sebmb.location_block lb JOIN sebmb.crime_location cl USING(lb_id)
JOIN sebmb.crime_record cr USING(cl_id)
GROUP BY (block,year)
HAVING year = 2012
ORDER BY count(*) DESC
LIMIT 10;

        block         | count | year
----------------------+-------+------
 042XX W CARMEN AVE   |   376 | 2012
 030XX N LINDER AVE   |   373 | 2012
 072XX N SHERIDAN RD  |   367 | 2012
 105XX S TRUMBULL AVE |   326 | 2012
 0000X E 108TH ST     |   319 | 2012
 113XX S OAKLEY AVE   |   318 | 2012
 112XX S NORMAL AVE   |   261 | 2012
 011XX W 104TH ST     |   258 | 2012
 069XX S EMERALD AVE  |   254 | 2012
 058XX N DRAKE AVE    |   228 | 2012
(10 rows)

In [ ]:
#these are the top 10 dates with the highest crime in descending order. This is helpful for Chicago 
#police because they get to see what months have the most activity. This appears that January has the
#highest amount of recorded crime in 2012 with the top 3 rows being different times on January 1st. 
#This could mean that crime is very high or they update their crime reports at the start of the new year. 

SELECT cr.crime_date, count(*)
FROM sebmb.crime_record cr
GROUP BY crime_date
ORDER BY count(*) DESC 
LIMIT 10;


  crime_date   | count
----------------+-------
 1/1/2012 0:01  |   153
 1/1/2012 0:00  |    98
 1/1/2012 9:00  |    79
 3/1/2012 9:00  |    67
 6/1/2012 9:00  |    66
 10/1/2012 9:00 |    63
 8/1/2012 9:00  |    59
 6/1/2012 12:00 |    58
 5/1/2012 9:00  |    57
 9/1/2012 9:00  |    55
(10 rows)

In [ ]:
#These are the top 5 most common crimes in chicago and what the crimes are. The highest are low level 
#crimes like theft, cannabis, battery, and vehicle damages. These are noted in the logs and are very 
#common occurances. This might help make a case when people are looking to legalize cannabis and they 
#can reference that 17,719 people were logged as committing a crime in 2012. These numbers can reduce criminal 
#activity when small charges like these are not seen as crimes.


SELECT cr.iucr, COUNT(*)
FROM sebmb.crime_record cr
GROUP BY iucr
ORDER BY count(*) DESC
LIMIT 5;

 iucr | count
------+-------
 820  | 29524
 486  | 28746
 460  | 20541
 1811 | 17719
 1320 | 16505
(5 rows)

SELECT * FROM sebmb.crime_code WHERE iucr IN ('820','486','460','1811','1320');

 iucr | fbi_code |         description          |  primary_type
------+----------+------------------------------+-----------------
 1811 | 18       | POSS: CANNABIS 30GMS OR LESS | NARCOTICS
 820  | 6        | $500 AND UNDER               | THEFT
 460  | 08B      | SIMPLE                       | BATTERY
 486  | 08B      | DOMESTIC BATTERY SIMPLE      | BATTERY
 1320 | 14       | TO VEHICLE                   | CRIMINAL DAMAGE
(5 rows)

In [ ]:
#I was interested in seeing what the B stood for at the end of the fbi_code, so I filtered the crime_code 
#by only the code that has the 'B'. I found that the code means that it involved some sort of battery. 
#These 6 crime_codes are very common among many of the crimes in this document, so the police will want to 
#know the meaning of this code too.

SELECT cc.description, cc.primary_type 
FROM sebmb.crime_code cc 
WHERE fbi_code = '08B';

 description              | primary_type
--------------------------------------+--------------
 AGG PO HANDS NO/MIN INJURY           | BATTERY
 OF UNBORN CHILD                      | BATTERY
 PRO EMP HANDS NO/MIN INJURY          | BATTERY
 AGG: HANDS/FIST/FEET NO/MINOR INJURY | BATTERY
 SIMPLE                               | BATTERY
 DOMESTIC BATTERY SIMPLE              | BATTERY
(6 rows)